In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
from collections import defaultdict

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns


class PySankeyException(Exception):
    pass


class NullsInFrame(PySankeyException):
    pass


class LabelMismatch(PySankeyException):
    pass


def check_data_matches_labels(labels, data, side):
    if len(labels > 0):
        if isinstance(data, list):
            data = set(data)
        if isinstance(data, pd.Series):
            data = set(data.unique().tolist())
        if isinstance(labels, list):
            labels = set(labels)
        if labels != data:
            msg = "\n"
            if len(labels) <= 20:
                msg = "Labels: " + ",".join(labels) + "\n"
            if len(data) < 20:
                msg += "Data: " + ",".join(data)
            raise LabelMismatch('{0} labels and data do not match.{1}'.format(side, msg))
def sankey(left, center , right, leftWeight=None,centerWeight = None, rightWeight=None, colorDict=None,
           leftLabels=None,centerLables=None,centerColor = False,rightLabels=None, aspect=4, rightColor=False,
           fontsize=14, figureName=None, closePlot=False):
    '''
    Make Sankey Diagram showing flow from left-->right

    Inputs:
        left = NumPy array of object labels on the left of the diagram
        right = NumPy array of corresponding labels on the right of the diagram
            len(right) == len(left)
        leftWeight = NumPy array of weights for each strip starting from the
            left of the diagram, if not specified 1 is assigned
        rightWeight = NumPy array of weights for each strip starting from the
            right of the diagram, if not specified the corresponding leftWeight
            is assigned
        colorDict = Dictionary of colors to use for each label
            {'label':'color'}
        leftLabels = order of the left labels in the diagram
        rightLabels = order of the right labels in the diagram
        aspect = vertical extent of the diagram in units of horizontal extent
        rightColor = If true, each strip in the diagram will be be colored
                    according to its left label
    Ouput:
        None
    '''
    if leftWeight is None:
        leftWeight = []
    if rightWeight is None:
        rightWeight = []
    if leftLabels is None:
        leftLabels = []
    if rightLabels is None:
        rightLabels = []
    if centerLables is None:
        centerLabels = []
    if centerWeight is None:
        centerWeight = []
        
    # Check weights
    if len(leftWeight) == 0:
        leftWeight = np.ones(len(left))
        
    if len(centerWeight) == 0:
        centerWeight = leftWeight

    if len(rightWeight) == 0:
        rightWeight = centerWeight
    

    plt.figure()
    plt.rc('text', usetex=False)
    plt.rc('font', family='serif')

    # Create Dataframe
    if isinstance(left, pd.Series):
        left = left.reset_index(drop=True)
    if isinstance(center, pd.Series):
        center = center.reset_index(drop=True)
    if isinstance(right, pd.Series):
        right = right.reset_index(drop=True)
    dataFrame = pd.DataFrame({'left': left, 'center':center,'right': right, 'leftWeight': leftWeight, 'centerWeight' : centerWeight,
                              'rightWeight': rightWeight}, index=range(len(left)))

    if len(dataFrame[(dataFrame.left.isnull()) | (dataFrame.center.isnull()) |(dataFrame.right.isnull())]):
        raise NullsInFrame('Sankey graph does not support null values.')

    # Identify all labels that appear 'left' or 'right'
    allLabels = pd.Series(np.r_[dataFrame.left.unique(),dataFrame.center.unique(), dataFrame.right.unique()]).unique()

    # Identify left labels
    if len(leftLabels) == 0:
        leftLabels = pd.Series(dataFrame.left.unique()).unique()
    else:
        check_data_matches_labels(leftLabels, dataFrame['left'], 'left')
    
    if len(centerLabels) == 0:
        centerLabels = pd.Series(dataFrame.center.unique()).unique()
    else:
        check_data_matches_labels(leftLabels, dataFrame['center'], 'center')

    # Identify right labels
    if len(rightLabels) == 0:
        rightLabels = pd.Series(dataFrame.right.unique()).unique()
    else:
        check_data_matches_labels(centerLabels, dataFrame['right'], 'right')
    # If no colorDict given, make one
    if colorDict is None:
        colorDict = {}
        palette = "hls"
        colorPalette = sns.color_palette(palette, len(allLabels))
        for i, label in enumerate(allLabels):
            colorDict[label] = colorPalette[i]
    else:
        missing = [label for label in allLabels if label not in colorDict.keys()]
        if missing:
            msg = "The colorDict parameter is missing values for the following labels : "
            msg += '{}'.format(', '.join(missing))
            raise ValueError(msg)

    # Determine widths of individual strips
    ns_l = defaultdict()
    ns_c = defaultdict()
    ns_r = defaultdict()
    for leftLabel in leftLabels:
        leftDict = {}
        rightDict = {}
        centerDict = {}
        
        for centerLabel in centerLabels:
            leftDict[centerLabel] = dataFrame[(dataFrame.left == leftLabel) & (dataFrame.center == centerLabel)].leftWeight.sum()
            centerDict[centerLabel] = dataFrame[(dataFrame.left == leftLabel) & (dataFrame.center == centerLabel)].centerWeight.sum()
        ns_l[leftLabel] = leftDict
        ns_c[leftLabel] = centerDict
        
    for centerLabel in centerLabels:
        rightDict = {}
        centerDict = {}
    
        for rightLabel in rightLabels:
            centerDict[centerLabel] = dataFrame[(dataFrame.center == centerLabel) & (dataFrame.right == rightLabel)].centerWeight.sum()
            rightDict[rightLabel] = dataFrame[(dataFrame.center == centerLabel) & (dataFrame.right == rightLabel)].rightWeight.sum()
        ns_c[centerLabel] = centerDict
        ns_r[centerLabel] = rightDict

    # Determine positions of left label patches and total widths
    leftWidths = defaultdict()
    for i, leftLabel in enumerate(leftLabels):
        myD = {}
        myD['left'] = dataFrame[dataFrame.left == leftLabel].leftWeight.sum()
        if i == 0:
            myD['bottom'] = 0
            myD['top'] = myD['left']
        else:
            myD['bottom'] = leftWidths[leftLabels[i - 1]]['top'] + 0.02 * dataFrame.leftWeight.sum()
            myD['top'] = myD['bottom'] + myD['left']
            topEdge = myD['top']
        leftWidths[leftLabel] = myD
        
    # Determine positions of center label patches and total widths 
    centerWidths = defaultdict()
    for i, centerLabel in enumerate(centerLabels):
        myD = {}
        myD['center'] = dataFrame[dataFrame.center == centerLabel].centerWeight.sum()
        if i == 0:
            myD['bottom'] = 0
            myD['top'] = myD['center']
        else:
            myD['bottom'] = centerWidths[centerLabels[i - 1]]['top'] + 0.02 * dataFrame.centerWeight.sum()
            myD['top'] = myD['bottom'] + myD['center']
            topEdge_c = myD['top']
        centerWidths[centerLabel] = myD

    # Determine positions of right label patches and total widths
    rightWidths = defaultdict()
    for i, rightLabel in enumerate(rightLabels):
        myD = {}
        myD['right'] = dataFrame[dataFrame.right == rightLabel].rightWeight.sum()
        if i == 0:
            myD['bottom'] = 0
            myD['top'] = myD['right']
        else:
            myD['bottom'] = rightWidths[rightLabels[i - 1]]['top'] + 0.02 * dataFrame.rightWeight.sum()
            myD['top'] = myD['bottom'] + myD['right']
            topEdge_r = myD['top']
        rightWidths[rightLabel] = myD

    # Total vertical extent of diagram
    xMax_lc = topEdge_c /aspect
    xMax_cr = topEdge_r / aspect

    # Draw vertical bars on left and right of each  label's section & print label
    for leftLabel in leftLabels:
        plt.fill_between(
            [-0.02 * xMax_lc, 1.02],
            2 * [leftWidths[leftLabel]['bottom']],
            2 * [leftWidths[leftLabel]['bottom'] + leftWidths[leftLabel]['left']],
            color=colorDict[leftLabel],
            alpha=0.99
        )
        plt.text(
            -0.05 * xMax_lc,
            leftWidths[leftLabel]['bottom'] + 0.5 * leftWidths[leftLabel]['left'],
            leftLabel,
            {'ha': 'right', 'va': 'center'},
            fontsize=fontsize
        )
    for centerLabel in centerLabels:
        plt.fill_between(
            [xMax_lc, 1.02 * xMax_lc], 2 * [centerWidths[centerLabel]['bottom']],
            2 * [centerWidths[centerLabel]['bottom'] + centerWidths[centerLabel]['center']],
            color=colorDict[centerLabel],
            alpha=0.99
        )
        plt.text(
            1.05 * xMax_lc,
            centerWidths[centerLabel]['bottom'] + 0.5 * centerWidths[centerLabel]['center'],
            centerLabel,
            {'ha': 'center', 'va': 'center'},
            fontsize=5
        )
    for rightLabel in rightLabels:
        plt.fill_between(
            [xMax_cr, 3.02 * xMax_cr], 2 * [rightWidths[rightLabel]['bottom']],
            2 * [rightWidths[rightLabel]['bottom'] + rightWidths[rightLabel]['right']],
            color=colorDict[rightLabel],
            alpha=0.99
        )
        plt.text(
            3.05 * xMax_cr,
            rightWidths[rightLabel]['bottom'] + 0.5 * rightWidths[rightLabel]['right'],
            rightLabel,
            {'ha': 'center', 'va': 'center'},
            fontsize=fontsize
        )
        
    for leftLabel in leftLabels:
        for center in centerLabels :
            labelColor = leftLabel
            if centerColor :
                labelColor = centerColor
                for rightLabel in rightLabels:
                    labelColor = centerColor
                    if rightColor:
                        labelColor = rightLabel
                    if len(dataFrame[(dataFrame.left == leftLabel) & (dataFrame.center == centerLabel) &
                                     (dataFrame.right == rightLabel)]) > 0:
                        # Create array of y values for each strip, half at left value,
                        # half at right, convolve
                        ys_d = np.array(50 * [leftWidths[leftLabel]['bottom']] + 50 * [centerWidths[centerLabel]['bottom']])
                        ys_d = np.convolve(ys_d, 0.05 * np.ones(20), mode='valid')
                        ys_d = np.convolve(ys_d, 0.05 * np.ones(20), mode='valid')
                        ys_u = np.array(50 * [leftWidths[leftLabel]['bottom'] + ns_l[leftLabel][centerLabel]] + 50 * [centerWidths[centerLabel]['bottom'] + ns_l[leftLabel][centerLabel]])
                        ys_u = np.convolve(ys_u, 0.05 * np.ones(20), mode='valid')
                        ys_u = np.convolve(ys_u, 0.05 * np.ones(20), mode='valid')

                        ys_r = np.array(50 * [centerWidths[centerLabel]['bottom']] + 50 * [rightWidths[rightLabel]['bottom']])
                        ys_r = np.convolve(ys_r, 0.05 * np.ones(20), mode='valid')
                        ys_r = np.convolve(ys_r, 0.05 * np.ones(20), mode='valid')
                        ys_p = np.array(50 * [centerWidths[centerLabel]['bottom'] + ns_c[centerLabel][rightLabel]] + 50 * [rightWidths[rightLabel]['bottom'] + ns_c[centerLabel][rightLabel]])
                        ys_p = np.convolve(ys_p, 0.05 * np.ones(20), mode='valid')
                        ys_p = np.convolve(ys_p, 0.05 * np.ones(20), mode='valid')

                        # Update bottom edges at each label so next strip starts at the right place
                        leftWidths[leftLabel]['bottom'] += ns_l[leftLabel][centerLabel]
                        centerWidths[centerLabel]['bottom'] += ns_c[leftLabel][centerLabel]
                        rightWidths[rightLabel]['bottom'] += ns_r[centerLabel][rightLabel]
                        plt.fill_between(
                            np.linspace(0, xMax_lc, len(ys_d)), ys_d, ys_u, alpha=0.65,
                            color=colorDict[labelColor]
                        
                        )
                        plt.fill_between(
                            np.linspace(0, xMax_cr, len(ys_r)), ys_r, ys_p, alpha=0.65,
                            color=colorDict[labelColor]
                        
                        )
                        

    plt.gca().axis('off')
    plt.gcf().set_size_inches(6, 6)
    fig = dict(data=[data_trace], layout=layout)
    py.iplot(fig, validate=False)
    plt.show(fig)
    if figureName != None:
        plt.savefig("{}.png".format(figureName), bbox_inches='tight', dpi=150)
    if closePlot:
        plt.close()


In [3]:
Check_sankey = pd.read_csv("C:/Users/varun.behl/Documents/Document_excel/Validation_dataset_100.csv")

In [4]:
Check_sankey.pivot(index = 'User_id' , columns = 'LOT' , values = 'Drug_mention')

ValueError: Index contains duplicate entries, cannot reshape

In [5]:
colors = ['#ff4000','#ff8000','#ffbf00','#ffff00','#bfff00','#80ff00','#40ff00','#00ff00','#00ff40','#00ff80','#00f790',
    '#00ffff','#00bfff','#0080ff','#0040ff','#0000ff','#4000ff','#8000ff','#bf00ff','#ff00ff','#ffcd55',
    '#ff0080','#ff0040','#ff0000','#000080','#008B8B','#6B8E23','#708090','#800000','#87CEEB','#C0C0C0','#00FFFF','#0000FF',
    '#0000A0', '#ADD8E6', '#800080', '#FFFF00', '#00FF00', '#FF00FF', '#C0C0C0', '#808080', '#000000','#FFA500',
    '#A52A2A', '#800000', '#008000','#808000']

In [6]:
sankey(left = Check_sankey[1],center= Check_sankey[2],right = Check_sankey[3],aspect=20, colorDict=colorDict,   fontsize=9, figureName="LOT_diagram" , closePlot=False)
    

KeyError: 1